In [13]:
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm
import pickle
#!pip install -U feel-it
import gensim
import torch
from feel_it import EmotionClassifier, SentimentClassifier
import numpy as np 
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
dtypes = {
    'Username' : str,
    'Tweet': str,
    'Reply to': str,
    'Participants' : object,
    'Hashtags': object,
    'Date': str,
    'nLikes': int,	
    'nReplies': int,	
    'nRetweets': int
}

In [15]:
data = []
for politician in ['conte', 'letta', 'meloni', 'salvini']:
    filename = '/content/drive/MyDrive/Broken Mirror/data/' + politician + '.json'
    data_ = pd.read_json(filename, dtype=dtypes)
    data.append(data_)
df = pd.concat(data)
    

In [16]:
df.Username.replace('GiuseppeConteIT', 'C', inplace=True)
df.Username.replace('EnricoLetta', 'L', inplace=True)
df.Username.replace('GiorgiaMeloni', 'M', inplace=True)
df.Username.replace('matteosalvinimi', 'S', inplace=True)
df.replace('None', np.nan, inplace=True) 
df.fillna(value=np.nan, inplace=True) 
df.reset_index(drop=True, inplace=True)

In [17]:
df.head()

,Username,Tweet,Reply to,Participants,Hashtags,Date,nLikes,nReplies,nRetweets
0,C,A partire da oggi scattano alcune novità che m...,NaN,NaN,[],2020-07-01 19:20:21,2153,955,333
1,C,Complimenti alla nostra @GDF per la maxi-opera...,NaN,NaN,[],2020-07-01 13:38:24,3577,317,472
2,C,Un bimbo per strada mi ha chiesto se riuscirò ...,NaN,NaN,[],2020-07-02 17:58:18,7711,2104,974
3,C,Una icona italiana abbraccia la transizione en...,NaN,NaN,[fiat500],2020-07-03 18:41:38,3382,422,441
4,C,Our hearts and minds today are more than ever ...,NaN,NaN,[independenceday],2020-07-04 17:58:48,3015,444,316


In [19]:
emotion_classifier = EmotionClassifier()
sentiment_classifier = SentimentClassifier()

In [20]:
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("MilaNLProc/feel-it-italian-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("MilaNLProc/feel-it-italian-sentiment")

def sentiment_class(tweet):
  sentence = tweet
  inputs = tokenizer(sentence, return_tensors="pt")

# Call the model and get the logits
  labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
  outputs = model(**inputs, labels=labels)
  loss, logits = outputs[:2]
  logits = logits.squeeze(0)

# Extract probabilities
  proba = torch.nn.functional.softmax(logits, dim=0)

# Unpack the tensor to obtain negative and positive probabilities
  negative, positive = proba
  negativeP = np.round(negative.item(), 4)
  positiveP = np.round(positive.item(), 4)
  print(f"Probabilities: Negative {np.round(negative.item(),4)} - Positive {np.round(positive.item(),4)}")
  return [negativeP, positiveP]

In [ ]:
df['neg'], df['pos'] = zip(*df['Tweet'].apply(sentiment_class))


In [22]:
tokenizer = AutoTokenizer.from_pretrained("MilaNLProc/feel-it-italian-emotion")

model = AutoModelForSequenceClassification.from_pretrained("MilaNLProc/feel-it-italian-emotion")

def emotion_class(tweet):
  sentence = tweet
  inputs = tokenizer(sentence, return_tensors="pt")

  # Call the model and get the logits
  labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
  outputs = model(**inputs, labels=labels)
  loss, logits = outputs[:2]
  logits = logits.squeeze(0)

  # Extract probabilities
  proba = torch.nn.functional.softmax(logits, dim=0)
  proba = np.array(proba.detach().numpy())

  # Obtain emotion probabilities (sorted)
  label_names = ['anger', 'fear', 'joy', 'sadness']
  ranking = np.argsort(proba)
  ranking = ranking[::-1]
  for i in range(proba.shape[0]):
      l = label_names[ranking[i]]
      s = proba[ranking[i]]
      print(f"{l} {np.round(float(s), 4)}")
  return label_names[ranking[0]]

In [ ]:
df["Emotion"] = df['Tweet'].apply(emotion_class)

In [24]:
def set_polarity(neg, pos):
    if neg>pos:
        value = "neg"
    elif neg < pos:
        value ="pos"
    else:
        value ="neutral"
    return myvalue
df['Polarity'] = df.apply(lambda x: set_polarity(x['neg'], x['pos']), axis=1)


In [28]:
df.drop(['pos', 'neg'], axis=1, inplace=True)

In [30]:
df.to_json('brokenmirror-sentiment.json')